In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.style.use('seaborn-v0_8-darkgrid')

# Modelling and Forecasting
# ==============================================================================
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.plot import plot_prediction_distribution
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_pinball_loss
from skforecast.exceptions import LongTrainingWarning

# Configuration
# ==============================================================================
import warnings
warnings.filterwarnings('once')

In [3]:
# Data download
# ==============================================================================
url = (
       'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
       'data/vic_elec.csv'
)
data = pd.read_csv(url, sep=',')

# Data preparation (aggregation at daily level)
# ==============================================================================
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%dT%H:%M:%SZ')
data = data.set_index('Time')
data = data.asfreq('30min')
data = data.sort_index()
data = data.drop(columns='Date')
data = data.resample(rule='D', closed='left', label ='right')\
       .agg({'Demand': 'sum', 'Temperature': 'mean', 'Holiday': 'max'})

data.head(3)

,Demand,Temperature,Holiday
Time,,,
2012-01-01,82531.745918,21.047727,True
2012-01-02,227778.257304,26.578125,True
2012-01-03,275490.988882,31.751042,True


In [4]:
# Split data into train-validation-test
# ==============================================================================
data = data.loc['2012-01-01 00:00:00': '2014-12-30 23:00:00']
end_train = '2013-12-31 23:59:00'
end_validation = '2014-9-30 23:59:00'
data_train = data.loc[: end_train, :].copy()
data_val   = data.loc[end_train:end_validation, :].copy()
data_test  = data.loc[end_validation:, :].copy()

print(
    f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}"
    f"  (n={len(data_train)})"
)
print(
    f"Validation dates : {data_val.index.min()} --- {data_val.index.max()}"
    f"  (n={len(data_val)})"
)
print(
    f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}"
    f"  (n={len(data_test)})"
)

Train dates      : 2012-01-01 00:00:00 --- 2013-12-31 00:00:00  (n=731)
Validation dates : 2014-01-01 00:00:00 --- 2014-09-30 00:00:00  (n=273)
Test dates       : 2014-10-01 00:00:00 --- 2014-12-30 00:00:00  (n=91)


In [5]:
# Create and train a ForecasterAutoreg
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(
                                 learning_rate = 0.01,
                                 max_depth     = 10,
                                 n_estimators  = 500,
                                 random_state  = 123
                             ),
                 lags      = 7
             )

forecaster.fit(y=data.loc[end_train:end_validation, 'Demand'])
forecaster

ForecasterAutoreg 
Regressor: LGBMRegressor(learning_rate=0.01, max_depth=10, n_estimators=500,
              random_state=123) 
Lags: [1 2 3 4 5 6 7] 
Transformer for y: None 
Transformer for exog: None 
Window size: 7 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: [Timestamp('2014-01-01 00:00:00'), Timestamp('2014-09-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Regressor parameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.01, 'max_depth': 10, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': 123, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': 'warn', 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0} 
fit_kwargs: {} 
C

In [6]:
# Predict 10 different forecasting sequences of 7 steps each using bootstrapping
# ==============================================================================
boot_predictions = forecaster.predict_bootstrapping(steps=7, n_boot=10)
boot_predictions

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9
2014-10-01,208783.097577,204764.999003,201367.203420,212280.661192,204622.100582,202889.246579,210078.609982,210151.742392,200990.348257,203137.232359
2014-10-02,212340.130913,208320.030667,217221.147681,209996.152477,218464.949614,212545.697772,212061.596039,216579.376162,226902.326444,212847.017644
2014-10-03,221380.131363,223890.630246,224625.204840,207260.018970,206099.826214,288939.369743,220085.831843,229514.853352,230476.360893,224059.729506
2014-10-04,209943.773973,212946.528553,200027.296792,203592.696069,194120.730680,262225.653280,212516.896678,222042.271003,215236.033624,222487.580478
2014-10-05,193372.607408,177195.227257,194351.934752,206606.909536,202654.435499,297740.735825,192454.576530,199372.141645,208527.690318,197830.624380
2014-10-06,199626.367544,203338.054671,207145.334747,208993.695577,204766.177714,258050.416951,184903.916795,201473.420885,204790.425542,183312.907014
2014-10-07,201546.003371,212477.910879,209639.468951,204102.852012,225036.945159,257847.869040,197235.769115,196990.134684,200007.936217,207116.299541


In [14]:
# Predict intervals for next 7 steps, quantiles 10th and 90th
# ==============================================================================
predictions = forecaster.predict_percentiles(
                  steps=7, 
                  percentiles=[0, 10, 50, 90, 100], 
                  n_boot=1000
              )
predictions

,p0,p10,p50,p90,p100
2014-10-01,150754.365690,195483.862851,204763.256450,214883.472075,280281.990506
2014-10-02,158719.737178,204160.738452,213986.374931,225623.750301,291686.628402
2014-10-03,169432.230340,212599.136391,224169.141974,237675.514362,301025.385030
2014-10-04,148400.111461,196863.841123,212271.676527,234950.293307,330994.117895
2014-10-05,139708.490626,185544.868289,198331.157660,225497.131653,302777.168068
2014-10-06,156623.431501,188334.486578,201144.102231,226720.998742,325859.112026
2014-10-07,142412.947710,191797.716429,205073.424934,231948.852094,327699.339381


In [15]:
# Predict intervals for next 7 steps, quantiles 10th and 90th
# ==============================================================================
predictions = forecaster.predict_interval(steps=7, interval=[10, 90], n_boot=1000)
predictions

,pred,lower_bound,upper_bound
2014-10-01,205723.923923,195483.862851,214883.472075
2014-10-02,215167.163121,204160.738452,225623.750301
2014-10-03,225144.443075,212599.136391,237675.514362
2014-10-04,211406.440681,196863.841123,234950.293307
2014-10-05,194848.766987,185544.868289,225497.131653
2014-10-06,201901.819903,188334.486578,226720.998742
2014-10-07,208648.526025,191797.716429,231948.852094


In [16]:
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog_predict

In [17]:
forecaster = ForecasterAutoreg(
                     regressor        = LinearRegression(),
                     lags             = 3,
                     transformer_y    = StandardScaler(),
                     transformer_exog = StandardScaler(),
                 )

forecaster.fit(y=y, exog=exog)
results = forecaster.predict_percentiles(
                steps               = 2,
                exog                = exog_predict,
                percentiles         = [5, 55, 95],
                n_boot              = 4,
                in_sample_residuals = True
            )
results

,p5,p55,p95
50,0.400576,0.501011,0.690248
51,-0.051655,0.059093,0.349575


In [18]:
results.values

array([[ 0.40057552,  0.50101108,  0.69024814],
       [-0.05165537,  0.05909311,  0.34957492]])